In [8]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import geopandas as gpd
import datetime
import os
import requests
import zipfile

if os.path.exists('BRUTOS/modis.ba.poly.shp'):
    pass
else:
    os.mkdir('BRUTOS')
    r = requests.get('https://maps.wild-fire.eu/effis?service=WFS&request=getfeature&typename=ms:modis.ba.poly&version=1.1.0&outputformat=SHAPEZIP')
    with open('BRUTOS/effis.zip', 'wb') as f:
        f.write(r.content)
    with zipfile.ZipFile("BRUTOS/effis.zip","r") as zip_file:
        zip_file.extractall("BRUTOS")

effis = gpd.read_file('BRUTOS/modis.ba.poly.shp')
effis = effis[effis.COUNTRY == 'ES']
effis['year'] = effis.apply(lambda x: x.FIREDATE.split('-')[0], axis=1)
effis = effis[effis.year == '2022']
effis['SUP'] = effis.apply(lambda x: int(x.AREA_HA), axis=1)
gif = effis[effis.SUP > 500]
gif['fecha'] = gif.apply(lambda x: x.FIREDATE.split(' ')[0], axis=1)
gif['f'] = gif.apply(lambda x: datetime.datetime.strptime(x.fecha, '%Y-%m-%d'), axis=1)
gif['fecha_fin'] = gif.apply(lambda x: x.LASTUPDATE.split(' ')[0], axis=1)
gif['fecha_fin'] = gif.apply(lambda x: datetime.datetime.strptime(x.fecha_fin, '%Y-%m-%d'), axis=1)
gif = gif[gif.f >= datetime.datetime.strptime('2022-06-01', '%Y-%m-%d')]
display(gif)

/home/nano/miniconda3/envs/pdalenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/nano/miniconda3/envs/pdalenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/home/nano/miniconda3/envs/pdalenv/lib/python3.10/site-packages/geopandas/geodataframe.py:1472: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

,id,FIREDATE,LASTUPDATE,COUNTRY,PROVINCE,COMMUNE,AREA_HA,BROADLEA,CONIFER,MIXED,...,ARTIFSURF,OTHERLC,PERCNA2K,CLASS,geometry,year,SUP,fecha,f,fecha_fin
47898,203908,2022-06-20 13:04:00,2022-06-20 21:42:00,ES,Teruel,Burbáguena,1796,12.625139043380832,10.456062291434346,0,...,0,0,0,FireSeason,"POLYGON ((-1.34379 41.03680, -1.34436 41.03678...",2022,1796,2022-06-20,2022-06-20,2022-06-20
49446,203643,2022-06-17 11:45:00,2022-06-17 13:59:00,ES,Toledo,Toledo,1738,61.01207590568942,0,0,...,0,0.4025301897642092,0,FireSeason,"POLYGON ((-4.10178 39.86295, -4.10229 39.86164...",2022,1738,2022-06-17,2022-06-17,2022-06-17
50428,208294,2022-07-15 02:11:00,2022-07-24 14:05:00,ES,Ourense,Vilariño de Conso,7090,8.652762119503823,4.749154453213011,0,...,0,0,99.99999999999983,FireSeason,"POLYGON ((-7.36136 42.18360, -7.36095 42.18381...",2022,7090,2022-07-15,2022-07-15,2022-07-24
51507,203085,2022-06-08 13:27:00,2022-06-11 14:12:00,ES,Málaga,Júzcar,5042,1.6636957813428073,53.00059417706372,4.1790453555158615,...,0,0,98.8651178278651,FireSeason,"POLYGON ((-5.14134 36.59670, -5.14075 36.59657...",2022,5042,2022-06-08,2022-06-08,2022-06-11
52687,206861,2022-07-19 12:19:00,2022-07-26 14:20:00,ES,Ciudad Real,Malagón,2509,8.624801271859752,8.386327503974229,1.7885532591414233,...,0,0,0,FireSeason,"POLYGON ((-4.06549 39.29856, -4.06622 39.29875...",2022,2509,2022-07-19,2022-07-19,2022-07-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61169,209382,2022-08-13 21:04:00,2022-08-19 01:15:00,ES,Alicante/Alacant,"Vall d'Ebo, la",12111,0,16.600725354434417,0.4615891856247901,...,0,0,0.15181272811120528,FireSeason,"POLYGON ((-0.11030 38.82538, -0.10930 38.82531...",2022,12111,2022-08-13,2022-08-13,2022-08-19
61484,211665,2022-10-29 12:57:00,2022-10-29 13:48:00,ES,Cantabria,Rionansa,531,28.83895131085602,0,0,...,0,0,0.010989957178075414,FireSeason,"POLYGON ((-4.35462 43.22085, -4.35483 43.22069...",2022,531,2022-10-29,2022-10-29,2022-10-29
61657,210488,2022-09-08 12:14:00,2022-09-17 12:44:00,ES,Granada,"Guájares, Los",5454,0,39.347268060138624,1.10011001100108,...,0,0,0,FireSeason,"POLYGON ((-3.57852 36.91314, -3.57839 36.91329...",2022,5454,2022-09-08,2022-09-08,2022-09-17
61683,209773,2022-08-24 22:25:00,2022-08-26 12:57:00,ES,Ourense,Lobeira,640,0,0,0,...,0,0,100,FireSeason,"POLYGON ((-8.07077 42.02088, -8.07073 42.02065...",2022,640,2022-08-24,2022-08-24,2022-08-26


In [ ]:
comunidades = gpd.read_file('../navarra/BRUTOS/COMUNIDADES/SHP_ETRS89/recintos_autonomicas_inspire_peninbal_etrs89/recintos_autonomicas_inspire_peninbal_etrs89.shp')

In [ ]:
inicio = datetime.datetime.strptime('2022-06-01', '%Y-%m-%d')
for i in range(154):
    fecha = inicio + datetime.timedelta(days=i)
    inc = gif[gif.f <= fecha]

    figure = plt.figure(figsize=(14,10))
    proj = ccrs.PlateCarree()
    ax = plt.subplot(projection=proj)

    minx, miny, maxx, maxy = comunidades.total_bounds
    ax.set_extent([minx, maxx, miny, maxy], crs=proj)
    comunidades.plot(ax=ax, edgecolor='black', facecolor=(0,0,1,0))
    for j, row in inc.iterrows():
        if row.fecha_fin >= fecha:
            ax.add_geometries([row.geometry], facecolor=(1,0,0,1), crs=proj)
            ax.plot(row.geometry.centroid.x, row.geometry.centroid.y, 'r', marker='.', markersize=50)
        else:
            ax.add_geometries([row.geometry], facecolor=(0,0,0,1), crs=proj)
    plt.savefig('MAPAS/mapa_{}.png'.format(fecha.strftime('%Y%m%d')), bbox_inches='tight')